In [ ]:
# |default_exp trajectory

In [ ]:
# |export
import logging

import numpy as np
import torch
from fastcore.basics import store_attr
from fastcore.foundation import L, Self

# Trajectory
> Helpers to compute trajectory points from a ghost

In [ ]:
# |export
def segment_dists(p, a, b):
    d = np.divide(b - a, np.expand_dims(np.linalg.norm(b - a, axis=1), 1))
    s = np.sum((a - p) * d, axis=1)
    t = np.sum((p - b) * d, axis=1)
    h = np.stack([s, t, np.zeros_like(t)]).max(0)
    c = np.cross(p - a, d)
    return np.hypot(h, np.linalg.norm(c, axis=1))


def segment_project(p, a, b):
    ap = p - a
    ab = b - a
    t = np.dot(ap, ab) / np.dot(ab, ab)
    t = max(0, min(1, t))
    result = a + t * ab
    return result

def aligned(v, a, b):
    return 0 <= np.dot(v, b - a)

In [ ]:
# |export
class Trajectory:
    def __init__(self, ghost):
        store_attr()
        self.samples = L(ghost["samples"])
        self.positions = np.array(
            list(self.samples.map(Self["position"])), dtype=np.float32
        )
        self.racetimes = np.array(list(self.samples.map(Self["racetime"])))
        self.checkpoints = np.array(ghost["checkpoints"])
        self.cp_ids = (
            self.checkpoints
            < np.tile(self.racetimes, (len(self.checkpoints), 1)).transpose()
        ).argmax(0)
        self.cp_ids[-1] = len(self.samples) - 1
        self.segment_as = self.positions[:-1]
        self.segment_bs = self.positions[1:]
        self.segment_norm = np.linalg.norm(self.segment_bs - self.segment_as, axis=1)
        self.segment_cumsum = self.segment_norm.cumsum()

    def observation(self, position, nb_cp, cp_offset, jump_z_coef=1):
        distances, distance, argmin, offset, last = self._distances(position, nb_cp, cp_offset)
        projection = segment_project(
            position, self.segment_as[argmin], self.segment_bs[argmin]
        )

        if (
            self.samples[argmin]["ground_contact"]
            or self.samples[argmin + 1]["ground_contact"]
        ):
            weighted_distance = distance
        else:
            weighted_distance = np.linalg.norm(
                (projection - position) * [1, jump_z_coef, 1]
            )

        travel = self.segment_cumsum[argmin - 1] if argmin else 0
        travel += np.linalg.norm(projection - self.segment_as[argmin])
        progress = travel / self.segment_cumsum[-1]
        missed_checkpoint = np.allclose(projection, self.segment_bs[last - 1])
        return dict(
            distance=distance.item(),
            weighted_distance=weighted_distance.item(),
            travel=travel.item(),
            total_travel=self.segment_cumsum[-1].item(),
            progress=progress.item(),
            nb_max_checkpoints=len(self.checkpoints),
            missed_checkpoint=missed_checkpoint,
        )

    def next_positions(self, position, nb_cp, cp_offset, nb_positions, spacing):
        distances, distance, argmin, offset, last = self._distances(position, nb_cp, cp_offset)
        progress = (
            self.segment_cumsum[argmin - 1]
            if 0 < argmin
            else np.array(0.0, dtype=np.float32)
        )
        projection = segment_project(
            position, self.segment_as[argmin], self.segment_bs[argmin]
        )
        progress += np.linalg.norm(projection - self.segment_as[argmin])
        positions_progresses = progress + np.arange(
            spacing, (nb_positions + 1) * spacing, spacing, dtype=np.float32
        )
        positions_progresses = positions_progresses.clip(0, self.segment_cumsum[-1])
        positions_progresses = np.tile(
            self.segment_cumsum, (len(positions_progresses), 1)
        ) - np.expand_dims(positions_progresses, 1)
        positions_argmins = (positions_progresses < 0).argmin(1) - 1
        positions_offsets = -positions_progresses[
            np.arange(len(positions_progresses)), positions_argmins
        ]
        a = self.segment_as[positions_argmins]
        b = self.segment_bs[positions_argmins]
        positions = (
            a
            + (b - a)
            * (positions_offsets / self.segment_norm[positions_argmins])[:, np.newaxis]
        )
        return positions

    def _distances(self, position, nb_cp, cp_offset):
        # last checkpoint is equivalent to the finish
        if nb_cp == len(self.checkpoints):
            nb_cp -= 1
        offset = self.cp_ids[nb_cp + cp_offset - 1] - 1 if 0 < nb_cp + cp_offset else 0
        last = self.cp_ids[nb_cp]
        distances = segment_dists(
            position,
            self.segment_as[offset:last],
            self.segment_bs[offset:last],
        )
        argmin = distances.argmin()
        distance = distances[argmin]
        argmin += offset
        return distances, distance, argmin, offset, last

    def frame_trajectory(
        self,
        position,
        rotation,
        nb_cp,
        cp_offset,
        nb_positions,
        spacing,
        device="cpu",
    ):
        targets = self.next_positions(position, nb_cp, cp_offset, nb_positions, spacing) - position
        result = (
            torch.from_numpy(rotation.apply(targets))
            .to(device, non_blocking=True)
            .float()
        )
        norm = torch.linalg.norm(result, dim=1).unsqueeze(1)
        linspace = (
            torch.linspace(spacing, nb_positions * spacing, nb_positions)
            .unsqueeze(1)
            .to(device, non_blocking=True)
        )
        return torch.cat([result / norm, norm / linspace], dim=1)

    def aligned(
        self, 
        position,
        nb_cp,
        cp_offset,
        velocity,
        device="cpu",
    ):
        distances, distance, argmin, offset, last = self._distances(position, nb_cp, cp_offset)
        result = aligned(velocity, self.segment_as[argmin], self.segment_bs[argmin])
        return torch.tensor([result], device=device)

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()